In [88]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft
from scipy import ndimage
from PIL import ImageOps


image_path = 'Js-IM-0039-0005.jpg'  # Replace with your image file

# Load the image
image = cv2.imread(image_path)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ndimage.measurements.center_of_mass(gray)


C:\Users\15741\AppData\Local\Temp\ipykernel_16160\711208980.py:16: DeprecationWarning: Please import `center_of_mass` from the `scipy.ndimage` namespace; the `scipy.ndimage.measurements` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.measurements.center_of_mass(gray)


(1627.2672986909547, 702.1355064921829)

In [ ]:

# Compute the Sobel gradient in x and y directions
grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)

# Compute the magnitude and angle of the gradients
magnitude = np.sqrt(grad_x**2 + grad_y**2)
angle = np.arctan2(grad_y, grad_x) * 180 / np.pi  # Convert to degrees

# Normalize angle to [0, 360]
angle[angle < 0] += 360
angle = angle.astype('float32')

angle = cv2.medianBlur(angle, 5)

plt.figure(figsize=(30, 10))
plt.hist(angle[angle > 0].ravel(), bins=720)
plt.show()

In [80]:
histogram, bin_edges = np.histogram(angle[angle > 0], bins=720, range=(0, 360))
hist_fft = fft(histogram)
convolution = ifft(hist_fft * np.conjugate(hist_fft)).real

plt.figure(figsize=(30, 10))
plt.show(histogram.ravel())
plt.show()

<Figure size 3000x1000 with 0 Axes>

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

<Figure size 3000x1000 with 0 Axes>

In [ ]:

# Function to compute the gradient of the image using the Sobel operator
def compute_gradient(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Compute the Sobel gradient in x and y directions
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    
    # Compute the magnitude and angle of the gradients
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    angle = np.arctan2(grad_y, grad_x) * 180 / np.pi  # Convert to degrees
    
    # Normalize angle to [0, 360]
    angle[angle < 0] += 360
    
    return magnitude, angle

# Function to compute the gradient orientation histogram
def gradient_orientation_histogram(angle, bins=360):
    histogram, bin_edges = np.histogram(angle, bins=bins, range=(0, 360))
    return histogram

# Fourier-based symmetry detection using the orientation histogram
def detect_symmetry(histogram):
    # Perform FFT on the histogram
    hist_fft = fft(histogram)
    
    # Convolve the histogram with itself using inverse FFT of the squared FFT
    convolution = ifft(hist_fft * np.conjugate(hist_fft)).real
    
    # Detect peaks in the convolution, which correspond to symmetry axes
    peaks = np.where(convolution > np.percentile(convolution, 90))[0]
    
    return peaks, convolution

# Function to visualize the symmetry axes on the original image
def draw_symmetry_lines(image, peaks):
    rows, cols = image.shape[:2]
    center = (cols // 2, rows // 2)
    
    # Convert peak angles to radians for drawing lines
    angles = np.deg2rad(peaks)
    
    # Draw symmetry axes
    for angle in angles:
        x = int(center[0] + np.cos(angle) * cols)
        y = int(center[1] + np.sin(angle) * rows)
        cv2.line(image, center, (x, y), (0, 255, 0), 2)
    
    return image

# Main function to process the image and detect symmetry
def symmetry_detection(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Step 1: Compute the gradient
    magnitude, angle = compute_gradient(image)
    
    # Step 2: Obtain the gradient orientation histogram
    histogram = gradient_orientation_histogram(angle)
    
    # Step 3: Detect symmetry using Fourier Transform
    peaks, convolution = detect_symmetry(histogram)
    
    # Step 4: Draw the detected symmetry lines on the image
    result_image = draw_symmetry_lines(image.copy(), peaks)
    
    # Plot the result
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    
    plt.subplot(1, 3, 2)
    plt.plot(histogram)
    plt.title("Gradient Orientation Histogram")
    
    plt.subplot(1, 3, 3)
    plt.plot(convolution)
    plt.title("Convolution Peaks for Symmetry")
    
    plt.show()
    
    # Show the image with detected symmetry lines
    cv2.imshow("Symmetry Detected", result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()